In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline 

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

In [ ]:
all_data = pd.concat([train, test], sort = False)
all_data.info()

In [ ]:
#Age, Fare의 null 값을 중앙값으로 치환
all_data['Age'] = all_data['Age'].fillna(all_data['Age'].median())
all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].median())

In [ ]:
all_data.info()

In [ ]:
sns.catplot(x='Embarked', kind='count', data=all_data);

In [ ]:
train['Survived'].value_counts()

In [ ]:
import matplotlib.font_manager as fm
', '.join(sorted([font.name for font in fm.fontManager.ttflist]))

In [ ]:
#내가 설치한 폰트
fontpaths = "/kaggle/input/nanumgothic" 
font_list = fm.findSystemFonts(fontpaths = fontpaths, fontext='ttf')
for font_file in font_list:
    fm.fontManager.addfont(font_file)
fm._load_fontmanager(try_read_cache=False)

In [ ]:
#내가 설치한 폰트를 기존 폰트들에 포함시키기
', '.join(sorted([font.name for font in fm.fontManager.ttflist]))

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family="NanumGothic")
#NanumGothic은 내가 설치한 폰트 중에서 확인 가능
#지금부터 plt를 이용하여 한글폰트를 사용가능

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14,5))
labels = ['사망', '생존']
 
# Pie Plot
train['Survived'].value_counts().plot.pie(ax=ax[0], explode=[0,0.1], shadow=True, autopct='%1.1f%%', labels=labels)
ax[0].set(ylabel='')
 
# Count Plot
sns.countplot(data=train, x='Survived', ax=ax[1])
ax[1].set(xlabel='', xticklabels=labels)
 
plt.show()

In [ ]:
#Embarked 중 가장 많은 's'로 null 값 치환
all_data['Embarked'] = all_data['Embarked'].fillna('s')
all_data.info()

In [ ]:
# 나이 feature 구간 나누기 
all_data.loc[all_data['Age'] <= 15, 'Age'] = 0
all_data.loc[(all_data['Age'] > 15) & (all_data['Age'] <= 25), 'Age'] = 1
all_data.loc[(all_data['Age'] > 25) & (all_data['Age'] <= 35), 'Age'] = 2
all_data.loc[(all_data['Age'] > 35) & (all_data['Age'] <= 45), 'Age'] = 3
all_data.loc[(all_data['Age'] > 45) & (all_data['Age'] <= 60), 'Age'] = 4
all_data.loc[ all_data['Age'] > 60, 'Age'] = 5

In [ ]:
# Title 추출 함수
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+\.)', name)
    
    if title_search:
        return title_search.group(1)
    return ""

In [ ]:
# Title 추출
all_data['Title'] =  all_data['Name'].apply(get_title)
all_data['Title'].value_counts()

In [ ]:
# Title을 대표 Title로 축소
all_data['Title'] = all_data['Title'].replace(['Capt.', 'Dr.', 'Major.', 'Rev.'], 'Officer.')
all_data['Title'] = all_data['Title'].replace(['Lady.', 'Countess.', 'Don.', 'Sir.', 'jonkheer.', 'Dona.'], 'Royal.')
all_data['Title'] = all_data['Title'].replace(['Mlle.', 'Ms.'], 'Miss.')
all_data['Title'] = all_data['Title'].replace(['Mme.'], 'Mrs.')
all_data['Title'].value_counts()

In [ ]:
# Cabin null값 처리
all_data['Cabin'] = all_data['Cabin'].fillna('Missing')
all_data['Cabin'] = all_data['Cabin'].str[0]
all_data['Cabin'].value_counts()

In [ ]:
# 가족 수 및 혼자사는지 여부에 대한 feature 추가
all_data['Family_Size'] = all_data['SibSp'] + all_data['Parch'] + 1
all_data['IsAlone'] = 0
all_data.loc[all_data['Family_Size']==1, 'IsAlone'] = 1
all_data.head()

In [ ]:
# 모델링에 필요 없는 feature 삭제
all_data.drop(['Name', 'Ticket'], axis=1, inplace=True)
all_data.head()

In [ ]:
# 번주형 feature 가변수화하기
all_dummies = pd.get_dummies(all_data, drop_first=True)
all_dummies.head()

In [ ]:
train = all_dummies[all_dummies['Survived'].notna()]
train.info()

In [ ]:
test = all_dummies[all_dummies['Survived'].isna()]
test.info()

In [ ]:
X = train.drop(['PassengerId', 'Survived'], axis=1)
y = train['Survived']

test = test.drop(['PassengerId', 'Survived'], axis=1)

In [ ]:
clf = RandomForestClassifier(n_estimators=200, min_samples_split=10, random_state=42)
clf.fit(X,y)

In [ ]:
pred = clf.predict(test).astype(int)
submission['Survived'] = pred

In [ ]:
submission.to_csv("submission.csv", index=False)